# __Использованные материалы__

* [LightFM Github](https://github.com/lyst/lightfm)
* [LightFM documentation](https://making.lyst.com/lightfm/docs/quickstart.html)
* [Google recommendation systems course](https://developers.google.com/machine-learning/recommendation)
* [Recommender system using Bayesian personalized ranking](https://towardsdatascience.com/recommender-system-using-bayesian-personalized-ranking-d30e98bba0b9)
* [Learning to Rank Sketchfab Models with LightFM](https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/)
* [How to build a Movie Recommender System in Python using LightFm](https://towardsdatascience.com/how-to-build-a-movie-recommender-system-in-python-using-lightfm-8fa49d7cbe3b)
* [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset/home?select=ratings_small.csv)

# __Краткое введение__

__LightFM__ - это реализация на Python'е ряда популярных алгоритмов рекомендаций, включая эффективную реализацию BPR и WARP. Он прост в использовании, быстр (благодаря многопоточности) и дает высококачественные результаты. <br>
Существуют две основные стратегии создания рекомендательных систем: 
* __Content-based Filtering__
* __Collaborative filtering__

На практике чаще всего они используются в совокупности.<br>
<em>Далее для удобства будет использоваться термин item, который подразумевает под собой сущности, рекомендуемые системой.</em>

### __Content-based Filtering__ 
Данный подход предполагает работу с метаданными пользователя, которые собираются различными способами:
* __explicit__ - пользователь заполняет анкеты для выявление предпочтений, к примеру оценивает какой-то item по дифференцированной шкале.<br>
* __implicit__ - все действия пользователя протоколируются для выявления предпочтений, к примеру переход по ссылками, информация о компьютере пользователя и тп.<br>

### __Collaborative filtering__ 
Данный подход использует группировку пользователей и item'ов по каким-то сходствам/критериям. Будет реализоваться следующая логика "Пользователям, которым понравился item $X$, также нравились item'ы $Y$". Похожесть как правило определяется следующими методами:<br>
* __Content-based__ - на основании характеристик item'ов и пользователей.<br>
* __Transaction-based__ - на основании того, входили ли item'ы в одну транзакцию, а пользователи совершали схожие действия.<br>

### Machine-learned ranking 
В __LightFM__ представлены два классических подхода MLR'а:
* __Bayesian Personalized Ranking (BLR)__ 
* __Weighted Approximate-Rank Pairwise (WARP)__ 

### Bayesian Personalized Ranking 
Основная идея заключается в выборке и попарном сравнение положительных и отрицательных item'ов. Алгоритм в упрощенном виде можно представить следующим образом:
1. Случайным образом возьмем пользователя $u$ и item $i$, который ранее был выбран пользователем, в таком случае item $i$ будет считаться <em>положительным.</em>
2. Случайным образом возьмем item $j$, который был выбран пользователем <em>реже</em>, чем $i$ (в том числе, который пользователь никогда не выбирал), в таком случае item $j$ будет считаться <em>отрицательным.</em>
3. Вычисляем оценку $p_{ui}$ и $p_{uj}$ пользователя $u$, а также положительного item'а $i$ и отрицательного item'а $j$ соответственно.
4. Находим разницу между положительными и отрицательными оценками, как $x_{uij} = p_{ui} - p_{uj}.$ 
5. Пропускаем эту разницу через сигмоид и используем ее для вычисления веса для обновления всех параметров модели с помощью Stochastic gradient descent(SGD).

### Weighted Approximate-Rank Pairwise
Концепция данного подхода схожа с BPR, за исключением случаев, когда происходит SGD обновление:
* В BPR SGD обновление происходит каждый раз с разницей в качестве веса.
* WARP запускает SGD обновление только в случае неверного предсказания (т.е. оценка отрицательного item'а больше положительного). Если предсказание было верным, то продолжаем выбирать отрицательные item'ы, пока не получим неверный прогноз или не достигнем некоторого порогового значения.

Для этих целей WARP предоставляет два гиперпараметра:
1. __Margin__ - определяет насколько ошибочным должен быть прогноз для совершения SGD обновления. 
2. __Cutoff__ - определяет сколько раз мы готовы выбирать отрицательные примеры, пытаясь получить неверное предсказание, прежде чем откажемся и перейдем к следующему пользователю.

<em>Автор статьи [Learning to Rank Sketchfab Models with LightFM](https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/) утверждает, что на практике вероятнее всего WARP предпочтительнее для большинства рекомендательных систем, нежели BPR.</em>

# __Тестовый пример__
Попробуем реализовать простейшую рекомендательную систему на основе [датасета, предоставляемого LightFM'ом.](https://grouplens.org/datasets/movielens/100k/)
### __Установка зависимостей__


In [1]:
!"c:\python37\python.exe" -m pip install --upgrade pip

In [4]:
!pip install lightfm

  Using cached lightfm-1.16.tar.gz (310 kB)
Using legacy 'setup.py install' for lightfm, since package 'wheel' is not installed.
    Running setup.py install for lightfm: started
    Running setup.py install for lightfm: finished with status 'done'
  Using cached lightfm-1.16.tar.gz (310 kB)
Using legacy 'setup.py install' for lightfm, since package 'wheel' is not installed.
    Running setup.py install for lightfm: started
    Running setup.py install for lightfm: finished with status 'done'


In [5]:
!pip install numpy

In [6]:
!pip install scipy

In [3]:
#Импорт необходимых библиотек для тестового примера
import numpy as np
from lightfm.datasets import fetch_movielens #метод lightfm для извлечения данных фильма
from lightfm import LightFM

In [55]:
#Получаем данные фильма с минимальным рейтингом 4
data = fetch_movielens(min_rating = 4.0)

#Отобразим обучающий и тестовый набор
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [30]:
#Создадим модель
model = LightFM(loss = 'warp')
#Тренировка
model.fit(data['train'], epochs=30, num_threads=2)

#Рекомендательная функция
def sample_recommendation(model, data, user_ids):
    #Число пользователей и фильмов в обучающем наборе
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
    	#Фильмы, которые уже понравились пользователям
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        #Предсказание фильмов, которые им понравится
        scores = model.predict(user_id, np.arange(n_items))
        #Сортирует результат по оценке
        top_items = data['item_labels'][np.argsort(-scores)]
        #Отображение результатов
        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)
            
sample_recommendation(model, data, [3, 25, 451])

User 3
     Known positives:
        Seven (Se7en) (1995)
        Contact (1997)
        Starship Troopers (1997)
     Recommended:
        L.A. Confidential (1997)
        Starship Troopers (1997)
        Cop Land (1997)
User 25
     Known positives:
        Dead Man Walking (1995)
        Star Wars (1977)
        Fargo (1996)
     Recommended:
        Fargo (1996)
        English Patient, The (1996)
        Contact (1997)
User 451
     Known positives:
        Twelve Monkeys (1995)
        Babe (1995)
        Mr. Holland's Opus (1995)
     Recommended:
        Raiders of the Lost Ark (1981)
        Amadeus (1984)
        Sting, The (1973)


## __Тестовый пример__